In [97]:
import pandas as pd
import requests
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bs
import urllib.request

import warnings
warnings.filterwarnings('ignore')

import re
from tqdm import tqdm 

# 인기 순위 30개 종목명 및 코드 추출
- 예시를 들기위해 2023_02_23을 기준으로 함
- 추후 크롤링 함수에 코드명을 일일이 넣는 것보다, 함수 매개변수를 자동으로 지정해주기 위한 작업
  - NS_users_crawler("252670",2) <- 여기서 "252670"와 같은 코드명
- 아래 df로 구핸해놓은 코드명 칼럼 변수만 뽑아서 반복하게끔 구현

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

from datetime import datetime

In [ ]:
url='https://finance.naver.com/sise/lastsearch2.naver'

resp = urllib.request.urlopen(url)
soup = BeautifulSoup(resp,'html.parser',from_encoding='euc-kr') # 불러올때 한글깨짐으로 encoding 설정

# a_tag = soup.find_all('a',{'class' : 'tltle'}) # class가 tltle인 a 태그를 찾기
# a_tag

In [ ]:
# 30개 잘 불러졌는지 확인 
print(len(a_tag))

In [ ]:
today = datetime.today()
print("현재 날짜 및 시간:", today)

count =0 
list = []

a_tag = soup.find_all('a',{'class' : 'tltle'})

for top30 in a_tag :
    only_name = top30.get_text()
    code_ = str(top30.get("href").split("=")[1]) # a 태그의 href 추출 후 코드명만 가져오기, str()로 저장
    count += 1
    list.append({"날짜":today.strftime('%Y-%m-%d'), "순위":count, "종목명":only_name, "코드명":code_})

df = pd.DataFrame(list)
print(df)

In [ ]:
df.to_csv("name_code_0206.csv", index=False) # csv로 저장

check_df = pd.read_csv("./name_code_0206.csv",  dtype={'코드명': str})
check_df

In [98]:
# 위에 만든 기업 1~30위 목록들을 가져온다.
check_df = pd.read_csv("./name_code_0206.csv",  dtype={'코드명': str}) # 코드명 앞에 0 유지
check_df

,날짜,순위,종목명,코드명
0,2023-02-06,1,삼성전자,005930
1,2023-02-06,2,카카오,035720
2,2023-02-06,3,SK하이닉스,000660
3,2023-02-06,4,타이거일렉,219130
4,2023-02-06,5,삼성바이오로직스,207940
5,2023-02-06,6,NAVER,035420
6,2023-02-06,7,서린바이오,038070
7,2023-02-06,8,마인즈랩,377480
8,2023-02-06,9,AP위성,211270
9,2023-02-06,10,KT&G,033780


In [99]:
check_df['종목명'].unique()

array(['삼성전자', '카카오', 'SK하이닉스', '타이거일렉', '삼성바이오로직스', 'NAVER', '서린바이오',
       '마인즈랩', 'AP위성', 'KT&G', '현대차', '포스코케미칼', '미투젠', '코오롱모빌리티그룹',
       '엔에프씨', '웹케시', 'LG화학', 'SK텔레콤', 'SKC', '한화솔루션', '한국조선해양', '삼성전자우',
       '링크제니시스', '위메이드', '진양홀딩스', '한미글로벌', '쏠리드', '콤텍시스템', 'HMM',
       'SK아이이테크놀로지'], dtype=object)

In [100]:
check_df.loc[0]

날짜     2023-02-06
순위              1
종목명          삼성전자
코드명        005930
Name: 0, dtype: object

In [101]:
check_df.loc[0][2]

'삼성전자'

In [102]:
check_df.loc[0][3]

'005930'

In [103]:
# 종토방의 게시물들과 정보를 가져오는 함수

def NS_users_crawler():
    # User-Agent 설정 : 네이버 보안때문에 유저처럼 설정해줘야 보임 
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    result_df = pd.DataFrame([])

    page_list = [800,115,60,1,18,78,1,60,90,9,50,110,3,107,2,3,14,30,12,26,14,50,4,180,1,31,9,28,120,37]


    for i in tqdm(range(1,2)): # 1~30위
        list = check_df.loc[i]
        
        last_page = page_list[i]
        
        url = "https://finance.naver.com/item/board.naver?code=%s&page=%s" % (list[3],str(last_page)) # list[3]은 기업명
        html = requests.get(url, headers=headers).content
        # 한글 깨짐 방지 decode
        soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
        last_page_element = soup.find("td", class_="on")
        last_page_number = int(last_page_element.a.text) # 가장 마지막 페이지


        for page in range(113, last_page_number+1):
            url = "https://finance.naver.com/item/board.naver?code=%s&page=%s" % (list[3], str(page)) # 해당 기업의 종토방

            # html → parsing
            html = requests.get(url, headers=headers).content
            # 한글 깨짐 방지 decode
            soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
            table = soup.find('table', {'class': 'type2'}) # 종목토론실 댓글 
            tb = table.select('tbody > tr') # 종목토론실 댓글 세부
            page = page
           
        

            for i in range(len(tb)):
                if len(tb[i].select('td > span')) > 0: # 글의 존재 여부
                    date = tb[i].select('td > span')[0].text[:10] # 글의 날짜
                    title = tb[i].select('td.title > a')[0]['title'] # 글의 제목

                    content_url = "https://finance.naver.com" + tb[i].select('td.title > a')[0]['href'] # 글 내용 읽기 위해 특정 게시물에 접속
                    content_html = requests.get(content_url, headers=headers).content
                    content_soup = bs(content_html.decode('euc-kr', 'replace'), 'html.parser')
                    content = content_soup.find('div', {'id': 'body'}).text # 글의 내용
                    
                    view = content_soup.find('span', {'class':'tah p11'}).text
                    agree = content_soup.find('strong',{'class':'tah p11 red01 _goodCnt'}).text
                    disagree = content_soup.find('strong',{'class':'tah p11 blue01 _badCnt'}).text

                    table = pd.DataFrame({'날짜': [date], '기업명':[list[2]], '제목+내용': [title + '  ' + content], '조회수' : [int(view)], '공감수' : [int(agree)], '비공감수' : [int(disagree)], '페이지' : [page]})
                    result_df = result_df.append(table)

                    # result_df.reset_index(inplace=True)
                    # result_df.drop(["index"],axis=1, inplace=True)

    result_df["날짜"] = pd.to_datetime(result_df['날짜'], format = "%Y.%m.%d")
    # result_df.set_index("날짜", inplace=True)

    return result_df


In [104]:
data = NS_users_crawler() # 몇 페이지까지 볼껀가 설정 (30개 기업 1페이지에 약 2분 정도 걸림)

data.to_csv('./week/카카오.csv')

100%|██████████| 1/1 [00:13<00:00, 13.73s/it]


In [105]:
data = data.reset_index(drop=True)

In [108]:
# 데이터를 저장

data = pd.read_csv('네이버_일주일치.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [123]:
# data의 날짜를 datetime으로 변경

data["날짜"] = pd.to_datetime(data['날짜'], format = "%Y.%m.%d")

In [124]:
# 특정 날짜 사이의 데이터만 추출하는 함수

def setting_data(start_date, end_date):
    global data
    mask = (data['날짜'] >= start_date) & (data['날짜'] <= end_date)
    data = data.loc[mask]

    return data

In [125]:
# 2월 1일부터 9일까지 추출

setting_data('2023-02-01', '2023-02-09')

,날짜,기업명,제목+내용,조회수,공감수,비공감수,페이지
3921,2023-02-09,삼성전자,그렇다면 ㅣ ㅎ ㅏ나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아...,419,0,0,197
3922,2023-02-09,삼성전자,5년안에 싹다 감옥소에 간다 누군지는 알것이다,229,7,2,197
3923,2023-02-09,삼성전자,나 콜옵 6장 넣었다규 ㆍ\rㆍ\rㆍ\r?\r ㆍ\rㆍ감사,220,0,0,197
3924,2023-02-09,삼성전자,개민주당 어리석은 것들 전기요금 가스요금 문제를 떠들 수록 느그가 불리하다\r돌대...,232,3,13,197
3925,2023-02-09,삼성전자,중국풍선 40개국이상 스파이짓속보 삼성게시판은\r 잡벌레집합소됫노ㅋ,212,2,0,197
...,...,...,...,...,...,...,...
40771,2023-02-01,SK아이이테크놀로지,고민되네 팔까말까\r20퍼면 많이 먹긴했는데,409,2,1,36
40772,2023-02-01,SK아이이테크놀로지,결국 파란색이네\r7만 왔을때 못팔았다\r다시 본전오면 팔께,410,1,1,36
40773,2023-02-01,SK아이이테크놀로지,곧 미국 투자발표나면 보자 쉽알공매들아 미국 분리막 독점이다,816,8,1,36
40774,2023-02-01,SK아이이테크놀로지,올해안에 10만원은 넘기자 ㅠ,841,3,4,36


In [128]:
# 데이터 저장

data.to_csv('네이버_0201_0209.csv')

In [143]:
data = data.reset_index(drop=True)

In [144]:
data

,날짜,기업명,제목+내용,조회수,공감수,비공감수,페이지
0,2023-02-09,삼성전자,그렇다면 ㅣ ㅎ ㅏ나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아...,419,0,0,197
1,2023-02-09,삼성전자,5년안에 싹다 감옥소에 간다 누군지는 알것이다,229,7,2,197
2,2023-02-09,삼성전자,나 콜옵 6장 넣었다규 ㆍ\rㆍ\rㆍ\r?\r ㆍ\rㆍ감사,220,0,0,197
3,2023-02-09,삼성전자,개민주당 어리석은 것들 전기요금 가스요금 문제를 떠들 수록 느그가 불리하다\r돌대...,232,3,13,197
4,2023-02-09,삼성전자,중국풍선 40개국이상 스파이짓속보 삼성게시판은\r 잡벌레집합소됫노ㅋ,212,2,0,197
...,...,...,...,...,...,...,...
30687,2023-02-01,SK아이이테크놀로지,고민되네 팔까말까\r20퍼면 많이 먹긴했는데,409,2,1,36
30688,2023-02-01,SK아이이테크놀로지,결국 파란색이네\r7만 왔을때 못팔았다\r다시 본전오면 팔께,410,1,1,36
30689,2023-02-01,SK아이이테크놀로지,곧 미국 투자발표나면 보자 쉽알공매들아 미국 분리막 독점이다,816,8,1,36
30690,2023-02-01,SK아이이테크놀로지,올해안에 10만원은 넘기자 ㅠ,841,3,4,36


In [132]:
# 전날 기준 상/하향 열을 추가해주는 코드

def NS_updown():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    updown_df = pd.DataFrame([])

    for i in tqdm(range(30)): # 기업1~30위

        list = check_df.loc[i]


        # 각 기업 별 주식 상/하향 사이트에서 가장 마지막 페이지를 추출해준다.
        # page=300으로 설정해놓으면 가장 마지막 페이지의 사이트로 이동한다.
        # 거기서 하단에 있는 가장 마지막 페이지 번호를 가져온다.

        url = "https://finance.naver.com/item/frgn.nhn?code=%s&page=%s" % (list[3],str(300)) # list[3]은 기업명
        html = requests.get(url, headers=headers).content
        # 한글 깨짐 방지 decode
        soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
        last_page_element = soup.find("td", class_="on")
        last_page_number = int(last_page_element.a.text) # 가장 마지막 페이지
        

        for page in range(1,last_page_number+1): # 각 기업의 시세 상/하락 사이트에서 page를 1 ~ 마지막 페이지까지 탐색

            date_list = [] # 상/하향 적용된 날짜를 담아두는 리스트
            updown_list = [] # 상/하향 여부를 담아두는 리스트     
            
            url = "https://finance.naver.com/item/frgn.nhn?code=%s&page=%s" % (list[3],str(page)) # 해당 기업의 일별 시세 정보방

            html = requests.get(url, headers=headers).content
            # 한글 깨짐 방지 decode
            soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
            table = soup.select_one("#content > div.section.inner_sub > table.type2")

            span_tags = table.select("span.tah.p10.gray03") # 해당 페이지의 상/하락 날짜의 개수
            
            # 가끔 가다가 상/하락이 아니라 그대로 유지인 경우가 있다.
            # 상/하락은 img정보에 상/하락 여부가 있어 img의 text를 추출하먄 상향/하향 구분되어 나오지만
            # 유지되는 경우는 lmg가 없어서 그대로 스킵되고 하나씩 밀리기 시작한다......
            # 그 때 스킵되는게 아니라 0(nan)값을 호출해줄 수 있도록 한다.
            rows = table.find_all("tr")
            for row in rows:
                td = row.find("td", {"class": "num"})
                if td and td["width"] == "67":
                    img = row.find("img")
                    if img:
                        updown_list.append(img.get("alt", ""))
                    else:
                        updown_list.append("0")
            
            #결과적으로 updown_list의 원소 개수와 span_tags(상/히락 일자 개수)는 동일해지게 된다.
            # 밑에서 for문으로 돌려 각자 생성된 list[i]를 적용해 호출

            # 상/하향 일자의 개수(num)만큼 반복한다.
            num_of_date = len(span_tags)
            num = num_of_date
            for i in range(num): 
                span_texts = [span.text.strip() for span in span_tags][i] # 주식 상/하락 기록된 날짜
                date_list.append(span_texts) 

                table = pd.DataFrame({'날짜': [date_list[i]], '기업명':[list[2]], '전날_대비_상승_하락' : [updown_list[i]]})
                updown_df = updown_df.append(table)

    updown_df["날짜"] = pd.to_datetime(updown_df['날짜'], format = "%Y.%m.%d")
    return updown_df


In [ ]:
# 날짜 / 기업명 / 상/하락 여부로 이루어진 dataframe을 생성한다. (이는 나중에 target_data로 활용)
# 또한 이 datefarme을 밑에서 원본 dataframe과 merge하여 원본 dataframe에 '전날 기준 상/하락 여부' 열을 추가한다.

data_updown = NS_updown() # 20분 정도 걸린다.

mapping = {'상승' : 1, '하락' : -1}

data_updown['전날_대비_상승_하락'] = data_updown['전날_대비_상승_하락'].map(mapping)

data_updown = data_updown.reset_index(drop=True)

data_updown['전날_대비_상승_하락'] = data_updown['전날_대비_상승_하락'].fillna(0)

data_updown['전날_대비_상승_하락'] = data_updown['전날_대비_상승_하락'].astype(int)

data_updown

In [23]:
data_updown['전날_대비_상승_하락'].unique()

array([-1,  1,  0])

In [24]:
# target_1.csv는 네이버 일일시세의 기업 별로 전체 모든 기간에 대한 일자 별 상/하락 여부를 긁어모은 것
# 네이버 일일시세에는 주말이나 공휴일은 포함되어있지 않아 해당 일은 skip되어 있다.

data_updown.to_csv('target_1.csv') # target_data를 저장

In [145]:
data_updown = pd.read_csv('target_1.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [146]:
data_updown

,날짜,기업명,전날_대비_상승_하락
0,2023-02-09,삼성전자,-1
1,2023-02-08,삼성전자,1
2,2023-02-07,삼성전자,1
3,2023-02-06,삼성전자,-1
4,2023-02-03,삼성전자,1
...,...,...,...
94887,2021-05-17,SK아이이테크놀로지,-1
94888,2021-05-14,SK아이이테크놀로지,-1
94889,2021-05-13,SK아이이테크놀로지,-1
94890,2021-05-12,SK아이이테크놀로지,-1


In [147]:
# 원본 데이터와 target data를 merge해 원본 데이터에 '전날 기준 상/하락 여부' 열을 추가 위해 복사본 생성

data_total = data.copy()

In [148]:
data_total

,날짜,기업명,제목+내용,조회수,공감수,비공감수,페이지
0,2023-02-09,삼성전자,그렇다면 ㅣ ㅎ ㅏ나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아...,419,0,0,197
1,2023-02-09,삼성전자,5년안에 싹다 감옥소에 간다 누군지는 알것이다,229,7,2,197
2,2023-02-09,삼성전자,나 콜옵 6장 넣었다규 ㆍ\rㆍ\rㆍ\r?\r ㆍ\rㆍ감사,220,0,0,197
3,2023-02-09,삼성전자,개민주당 어리석은 것들 전기요금 가스요금 문제를 떠들 수록 느그가 불리하다\r돌대...,232,3,13,197
4,2023-02-09,삼성전자,중국풍선 40개국이상 스파이짓속보 삼성게시판은\r 잡벌레집합소됫노ㅋ,212,2,0,197
...,...,...,...,...,...,...,...
30687,2023-02-01,SK아이이테크놀로지,고민되네 팔까말까\r20퍼면 많이 먹긴했는데,409,2,1,36
30688,2023-02-01,SK아이이테크놀로지,결국 파란색이네\r7만 왔을때 못팔았다\r다시 본전오면 팔께,410,1,1,36
30689,2023-02-01,SK아이이테크놀로지,곧 미국 투자발표나면 보자 쉽알공매들아 미국 분리막 독점이다,816,8,1,36
30690,2023-02-01,SK아이이테크놀로지,올해안에 10만원은 넘기자 ㅠ,841,3,4,36


In [155]:
data_total["날짜"] = data_total["날짜"].astype(str)

In [156]:
# merge할 때 datetime 있으면 안됨.
data_total["날짜"] = data_total["날짜"].str.replace("-", "").astype(int)

In [157]:
data_updown["날짜"] = data_updown["날짜"].str.replace("-", "").astype(int)

In [158]:
# 주식 상/하락 기록된 data_updown과 기존 데이터를 복사했던 data_total을 merge한다.

 # date_total에는 없고, data_updown에는 있는 날짜들도 전부 추가시키고 그 날짜들의 다른 열의 값들은 nan값이 채워진다.
data_merge = pd.merge(data_total, data_updown, on = ['날짜','기업명'], how = 'outer')

# 상/하락 열에서 생성된 빈 날짜 별 값들은 0으로 채워준다.
data_merge['전날_대비_상승_하락'] = data_merge['전날_대비_상승_하락'].fillna(0)

# nan값이 포함된 행을 전부 삭제한다. 이 과정에서 data_total에 새로 추가된 날짜들이 삭제된다.
data_merge = data_merge.dropna(axis=0)

# 값이 float화 되서 int로 바꾼다.
data_merge['전날_대비_상승_하락'] = data_merge['전날_대비_상승_하락'].astype(int)
data_merge['조회수'] = data_merge['조회수'].astype(int)
data_merge['공감수'] = data_merge['공감수'].astype(int)
data_merge['비공감수'] = data_merge['비공감수'].astype(int)
data_merge['페이지'] = data_merge['페이지'].astype(int)

# 다시 날짜를 datetime 형식으로 변경
data_merge['날짜'] = data_merge['날짜'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

# 제목 + 내용에서 '\r' 제거
data_merge["제목+내용"] = data_merge["제목+내용"].str.replace("\r", "")

In [159]:
data_merge.isnull().sum()

날짜             0
기업명            0
제목+내용          0
조회수            0
공감수            0
비공감수           0
페이지            0
전날_대비_상승_하락    0
dtype: int64

In [160]:
data_merge['전날_대비_상승_하락'].unique()

array([-1,  1,  0])

In [161]:
data_merge

,날짜,기업명,제목+내용,조회수,공감수,비공감수,페이지,전날_대비_상승_하락
0,2023-02-09,삼성전자,그렇다면 ㅣ ㅎ ㅏ나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아...,419,0,0,197,-1
1,2023-02-09,삼성전자,5년안에 싹다 감옥소에 간다 누군지는 알것이다,229,7,2,197,-1
2,2023-02-09,삼성전자,나 콜옵 6장 넣었다규 ㆍㆍㆍ? ㆍㆍ감사,220,0,0,197,-1
3,2023-02-09,삼성전자,개민주당 어리석은 것들 전기요금 가스요금 문제를 떠들 수록 느그가 불리하다돌대가리...,232,3,13,197,-1
4,2023-02-09,삼성전자,중국풍선 40개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노ㅋ,212,2,0,197,-1
...,...,...,...,...,...,...,...,...
30687,2023-02-01,SK아이이테크놀로지,고민되네 팔까말까20퍼면 많이 먹긴했는데,409,2,1,36,1
30688,2023-02-01,SK아이이테크놀로지,결국 파란색이네7만 왔을때 못팔았다다시 본전오면 팔께,410,1,1,36,1
30689,2023-02-01,SK아이이테크놀로지,곧 미국 투자발표나면 보자 쉽알공매들아 미국 분리막 독점이다,816,8,1,36,1
30690,2023-02-01,SK아이이테크놀로지,올해안에 10만원은 넘기자 ㅠ,841,3,4,36,1


In [162]:
# '\r'제거도 추가됨 원본 + 전날기준상하락

data_merge.to_csv('네이버_60페이지_상승하락적용.csv')

In [163]:
data_merge = pd.read_csv('네이버_60페이지_상승하락적용.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [165]:
data_merge.to_csv('네이버_0201_0209_상승하락적용.csv')

In [170]:
data_merge = pd.read_csv('네이버_0201_0209_상승하락적용.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [171]:
data_merge[data_merge['기업명']=='카카오']['날짜'].unique()

array(['2023-02-09', '2023-02-08', '2023-02-07', '2023-02-06',
       '2023-02-05', '2023-02-04', '2023-02-03', '2023-02-02',
       '2023-02-01'], dtype=object)

In [77]:
data.isnull().sum()

날짜             0
기업명            0
제목+내용          0
조회수            0
공감수            0
비공감수           0
페이지            0
전날_대비_상승_하락    0
dtype: int64